### Etymology Prefix Suffix Analysis

In [33]:
import pandas as pd
import numpy as np
import glob
import re

In [34]:
df_word_plus_nisanyan = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/Nisanyan Words Contain Plus.xlsx")
df_word_plus_nisanyan

,term,related_lang,related_term
0,+ark,Fransızca,+arque
1,+arşi,Fransızca,+archie
2,+faj,Fransızca ve İngilizce,+phage
3,+gam,Fransızca,+game
4,+gen,Fransızca,+gone
...,...,...,...
150,tri+,Fransızca ve İngilizce,tri+
151,trop(o)+,Fransızca ve İngilizce,tropo+
152,üni+,Fransızca ve İngilizce,uni+
153,vaz(o)+,Fransızca ve İngilizce,vaso+


In [35]:
def remove(x):
    try:
        x = x.replace("\n","")
        x = x.replace("\r","")
        x = x.strip()
    except:
        x = x
    return x

In [36]:
df_word_plus_nisanyan["term"] = df_word_plus_nisanyan["term"].apply(lambda x: remove(x))
df_word_plus_nisanyan["related_lang"] = df_word_plus_nisanyan["related_lang"].apply(lambda x: remove(x))
df_word_plus_nisanyan["related_term"] = df_word_plus_nisanyan["related_term"].apply(lambda x: remove(x))
df_word_plus_nisanyan

,term,related_lang,related_term
0,+ark,Fransızca,+arque
1,+arşi,Fransızca,+archie
2,+faj,Fransızca ve İngilizce,+phage
3,+gam,Fransızca,+game
4,+gen,Fransızca,+gone
...,...,...,...
150,tri+,Fransızca ve İngilizce,tri+
151,trop(o)+,Fransızca ve İngilizce,tropo+
152,üni+,Fransızca ve İngilizce,uni+
153,vaz(o)+,Fransızca ve İngilizce,vaso+


In [37]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Result/Word/Merge/Word_Merge2.xlsx")
#df_word = df_word.head(10000)
df_word

,word,frequency
0,bir,18835735
1,fikret,12101580
2,bu,11062659
3,ne,8025880
4,ve,7766036
...,...,...
164296,buldugumuzda,43
164297,boşandıklarını,42
164298,endişenlenme,42
164299,uzaylıymışım,42


In [38]:
disable_term_list = ["+ark","+gen","+gon","di+","+men","deka+","desi+","bis+","end(o)+","fil(o)+","mili+","ö+","or(o)+","pan(to)+","poli+","port+","post+","vaz(o)+"]

In [39]:
set_plus_term = set(df_word_plus_nisanyan.loc[:,"term"].values.tolist())
set_disable_term = set(disable_term_list)

In [40]:
df_term_diff = pd.DataFrame(set_plus_term.difference(set_disable_term), columns=["term"])
df_term_diff

,term
0,sit(o)+
1,hip(o)+2
2,pat(o)+
3,andr(o)+
4,hister(o)+
...,...
133,+metre
134,elektr(o)+
135,kript(o)+
136,opt(o)+


In [41]:
word_list = df_term_diff.loc[:,"term"].values.tolist()

In [42]:
df_word_result = pd.DataFrame()
for i in word_list:
    if i.startswith("+"):
        j = i.lstrip("+")
        word_in_video = df_word[df_word.word.str.contains(fr"{j}(?:$)", na=True)]  # string+ext=> word
        word_in_video.insert(0,"term",i)
    elif i.endswith("+"):
        j = i.rstrip("+")
        word_in_video = df_word[df_word.word.str.contains(fr"(?:^){j}", na=True)]  # ext+string=> word
        word_in_video.insert(0,"term",i)
    else:
        pass
    #word_in_video.insert(0,"word_ext",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
#df_word_result = df_word_result.sort_values(by="frequency", ascending=False)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result   

<ipython-input-42-58f65794987f>:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  word_in_video = df_word[df_word.word.str.contains(fr"(?:^){j}", na=True)]  # ext+string=> word


,term,word,frequency
0,pat(o)+,patolojik,957
1,pat(o)+,patoloji,744
2,pat(o)+,patojen,437
3,pat(o)+,patolog,356
4,pat(o)+,patojeni,154
...,...,...,...
1195,kript(o)+,kriptonlu,422
1196,kript(o)+,kriptondan,106
1197,kript(o)+,kriptonitin,73
1198,kript(o)+,kriptografi,67


In [43]:
df_pre_suff_nis_merge = pd.merge(df_word_result,df_word_plus_nisanyan, how="inner", on="term")
df_pre_suff_nis_merge

,term,word,frequency,related_lang,related_term
0,pat(o)+,patolojik,957,Fransızca ve İngilizce,patho+
1,pat(o)+,patoloji,744,Fransızca ve İngilizce,patho+
2,pat(o)+,patojen,437,Fransızca ve İngilizce,patho+
3,pat(o)+,patolog,356,Fransızca ve İngilizce,patho+
4,pat(o)+,patojeni,154,Fransızca ve İngilizce,patho+
...,...,...,...,...,...
1195,kript(o)+,kriptonlu,422,Fransızca ve İngilizce,crypto+
1196,kript(o)+,kriptondan,106,Fransızca ve İngilizce,crypto+
1197,kript(o)+,kriptonitin,73,Fransızca ve İngilizce,crypto+
1198,kript(o)+,kriptografi,67,Fransızca ve İngilizce,crypto+


In [44]:
df_pre_suff_nis_merge.word.nunique()

1092

In [45]:
df_pre_suff_nis_merge.to_excel("Nisanyan_Prefix_Suffix_Term_In_Word.xlsx", index=False)

##### SparkNLP Lemma

In [49]:
df_spark_lemma = pd.read_json("wR3tE0i-zc6Pl8revsMq4.json")  # lemma result
df_spark_lemma = df_spark_lemma.loc[:,["lem","token"]]
df_spark_lemma.rename(columns={"token":"word"}, inplace=True)
df_spark_lemma

,lem,word
0,patolojik,patolojik
1,patoloji,patoloji
2,patojen,patojen
3,patolog,patolog
4,patojen,patojeni
...,...,...
1196,kripton,kriptondan
1197,kriptonitin,kriptonitin
1198,kriptografi,kriptografi
1199,topografik,topografik


In [50]:
df_spark_lemma.lem.nunique()

522

In [51]:
df_pre_suff_nis_merge = pd.read_excel("Nisanyan_Prefix_Suffix_Term_In_Word.xlsx")
df_pre_suff_nis_merge

,term,word,frequency,related_lang,related_term
0,pat(o)+,patolojik,957,Fransızca ve İngilizce,patho+
1,pat(o)+,patoloji,744,Fransızca ve İngilizce,patho+
2,pat(o)+,patojen,437,Fransızca ve İngilizce,patho+
3,pat(o)+,patolog,356,Fransızca ve İngilizce,patho+
4,pat(o)+,patojeni,154,Fransızca ve İngilizce,patho+
...,...,...,...,...,...
1195,kript(o)+,kriptonlu,422,Fransızca ve İngilizce,crypto+
1196,kript(o)+,kriptondan,106,Fransızca ve İngilizce,crypto+
1197,kript(o)+,kriptonitin,73,Fransızca ve İngilizce,crypto+
1198,kript(o)+,kriptografi,67,Fransızca ve İngilizce,crypto+


In [53]:
df_lemma_merge = pd.merge(df_pre_suff_nis_merge,df_spark_lemma, how="inner", on="word")
df_lemma_merge.drop_duplicates(inplace=True)
df_lemma_merge

,term,word,frequency,related_lang,related_term,lem
0,pat(o)+,patolojik,957,Fransızca ve İngilizce,patho+,patolojik
1,pat(o)+,patoloji,744,Fransızca ve İngilizce,patho+,patoloji
3,+loji,patoloji,744,Fransızca,+logie,patoloji
5,pat(o)+,patojen,437,Fransızca ve İngilizce,patho+,patojen
7,+jen,patojen,437,Fransızca,+gène,patojen
...,...,...,...,...,...,...
1415,kript(o)+,kriptonlu,422,Fransızca ve İngilizce,crypto+,kripton
1416,kript(o)+,kriptondan,106,Fransızca ve İngilizce,crypto+,kripton
1417,kript(o)+,kriptonitin,73,Fransızca ve İngilizce,crypto+,kriptonitin
1418,kript(o)+,kriptografi,67,Fransızca ve İngilizce,crypto+,kriptografi


In [55]:
df_lemma_merge.drop_duplicates(subset="lem", inplace=True)
df_lemma_merge.reset_index(drop=True, inplace=True)
df_lemma_merge

,term,word,frequency,related_lang,related_term,lem
0,pat(o)+,patolojik,957,Fransızca ve İngilizce,patho+,patolojik
1,pat(o)+,patoloji,744,Fransızca ve İngilizce,patho+,patoloji
2,pat(o)+,patojen,437,Fransızca ve İngilizce,patho+,patojen
3,pat(o)+,patolog,356,Fransızca ve İngilizce,patho+,patolog
4,andr(o)+,android,1838,Fransızca ve İngilizce,andro+,android
...,...,...,...,...,...,...
517,kript(o)+,kripton,947,Fransızca ve İngilizce,crypto+,kripton
518,kript(o)+,kriptonit,785,Fransızca ve İngilizce,crypto+,kriptonit
519,kript(o)+,kriptonitin,73,Fransızca ve İngilizce,crypto+,kriptonitin
520,kript(o)+,kriptografi,67,Fransızca ve İngilizce,crypto+,kriptografi


In [56]:
df_lemma_merge.to_excel("Nisanyan_Prefix_Suffix_Term_In_Word_Lemma_Process.xlsx", index=False)